In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PE dosyaları **exe, dll, sys/drv, ocx, cp, scr** uzantılarından herhangi birine sahip Windows da Portable executable dosyalarını ifade etmektedir.

*    **cpl** : Control Panel
*    **dll** : Dynamic Link Library
*    **exe** : Executable File
*    **ocx** : ActiveX Control
*    **scr** : ScreenSaver
*    **sys/drv** : System File / Kernel Driver




In [2]:
!pip3 install pefile

     |████████████████████████████████| 71kB 1.9MB/s 
  Created wheel for pefile: filename=pefile-2019.4.18-cp36-none-any.whl size=60824 sha256=15b03f3c38f34c7ecefb13e1017015a78ed86ffc9fedbeea730d540623a7c24d
  Stored in directory: /root/.cache/pip/wheels/1c/a1/95/4f33011a0c013c872fe6f0f364dc463a2588120820e40a30d8
Successfully built pefile


Virtual Address : dosya hafızaya yüklendiği ve tüm hizalamaların yapıldığı durumda elimizde olan bir elemanın adresine verilen isimdir.

Relative Virtual Address : dosya belleğe yüklendikten sonra dosya içerisindeki belli bir bölümün yükleme adresinden ne kadar uzakta olacağını belirtir. PE de ImageBase'in belirttiği değerdir.

Program hafızaya yüklendikten sonraki başlangıç adresine ekleyeceğiniz RVA elemanın Virtual Address vermektedir.

> Virtual Address  = ImageBase + Relative Virtual Address

Relative Virtual Address dosya hafızada bulunduğu zaman geçerlidir.
Diste bulunan dosya için Relative Virtual Address değeri Raw Offset'e dönüştürülmelidir. 

#### **PE Dosya Yapısı**

PE dosyaları, headers ve sections dan oluşuyor.


DOS Header uyumluluk sağlamak için bulunuyor.



In [3]:
pefilePath="/content/drive/My Drive/PEFile/AdobeUpdater.exe"

In [4]:
import pefile
pe = pefile.PE(pefilePath)
pe.print_info()

----------DOS_HEADER----------

[IMAGE_DOS_HEADER]
0x0        0x0   e_magic:                       0x5A4D    
0x2        0x2   e_cblp:                        0x90      
0x4        0x4   e_cp:                          0x3       
0x6        0x6   e_crlc:                        0x0       
0x8        0x8   e_cparhdr:                     0x4       
0xA        0xA   e_minalloc:                    0x0       
0xC        0xC   e_maxalloc:                    0xFFFF    
0xE        0xE   e_ss:                          0x0       
0x10       0x10  e_sp:                          0xB8      
0x12       0x12  e_csum:                        0x0       
0x14       0x14  e_ip:                          0x0       
0x16       0x16  e_cs:                          0x0       
0x18       0x18  e_lfarlc:                      0x40      
0x1A       0x1A  e_ovno:                        0x0       
0x1C       0x1C  e_res:                         
0x24       0x24  e_oemid:                       0x0       
0x26       0x26

 winnt.h dosyasında DOS başlığının tanımlanması:

  

    typedef struct _IMAGE_DOS_HEADER {
    // DOS .EXE header
    USHORT e_magic;         // Magic number
    USHORT e_cblp;          // Bytes on last page of file
    USHORT e_cp;            // Pages in file
    USHORT e_crlc;          // Relocations
    USHORT e_cparhdr;       // Size of header in paragraphs
    USHORT e_minalloc;      // Minimum extra paragraphs needed
    USHORT e_maxalloc;      // Maximum extra paragraphs needed
    USHORT e_ss;            // Initial (relative) SS value
    USHORT e_sp;            // Initial SP value
    USHORT e_csum;          // Checksum
    USHORT e_ip;            // Initial IP value
    USHORT e_cs;            // Initial (relative) CS value
    USHORT e_lfarlc;        // File address of relocation table
    USHORT e_ovno;          // Overlay number
    USHORT e_res[4];        // Reserved words
    USHORT e_oemid;         // OEM identifier (for e_oeminfo)
    USHORT e_oeminfo;       // OEM information; e_oemid specific
    USHORT e_res2[10];      // Reserved words
    LONG   e_lfanew;        // File address of new exe header
    } IMAGE_DOS_HEADER, *PIMAGE_DOS_HEADER;


* **e_magic** : Dosyanın DOS olduğunu belirtmek için kullanılıyor.
* **e_lfanew** : PE başlığının offset değerini veriyor.

In [5]:
import pefile
pe = pefile.PE(pefilePath)
print("e_magic : " + hex(pe.DOS_HEADER.e_magic)) 
print("e_lfnew : " + hex(pe.DOS_HEADER.e_lfanew))

e_magic : 0x5a4d
e_lfnew : 0x100


PE Headerın(NT_HEADER) da doğrulamak için kullanılan ve PE imzası bulunmaktadır

In [6]:
import pefile
pe = pefile.PE(pefilePath)
print("Signature : " + hex(pe.NT_HEADERS.Signature))

Signature : 0x4550


##### Image File Header:

 PE dosyasına ait önemli bilgileri tutmaktadır.

    typedef struct _IMAGE_FILE_HEADER {
    USHORT  Machine;
    USHORT  NumberOfSections;
    ULONG   TimeDateStamp;
    ULONG   PointerToSymbolTable;
    ULONG   NumberOfSymbols;
    USHORT  SizeOfOptionalHeader;
    USHORT  Characteristics;
    } IMAGE_FILE_HEADER, *PIMAGE_FILE_HEADER;


* **Machine** : Programın çalışması beklenen CPU

* **NumberOfSections** : Section sayısı

* **TimeDateStamp** : Dosyanın üretildiği zaman

* **PointerToSymbolTable** : COFF sembol tablosuna gösterici 

* **NumberOfSymbols** : Sembol tablosundaki sembol sayısı

* **Characteristics** : Dosya hakkında bilgi

In [7]:
import pefile
pe = pefile.PE(pefilePath)

print("Machine : " + hex(pe.FILE_HEADER.Machine))

if hex(pe.FILE_HEADER.Machine) == '0x14c':
    print("This is a 32-bit binary")
else:
    print("This is a 64-bit binary")

print("TimeDateStamp : " + pe.FILE_HEADER.dump_dict()['TimeDateStamp']['Value'].split('[')[1][:-1]
)
print("NumberOfSections : " + hex(pe.FILE_HEADER.NumberOfSections))


Machine : 0x14c
This is a 32-bit binary
TimeDateStamp : Wed Aug  6 09:43:32 2008 UTC
NumberOfSections : 0x4


##### Optional Header

    typedef struct _IMAGE_OPTIONAL_HEADER {
    //
    // Standard fields.
    //
    USHORT  Magic;
    UCHAR   MajorLinkerVersion;
    UCHAR   MinorLinkerVersion;
    ULONG   SizeOfCode;
    ULONG   SizeOfInitializedData;
    ULONG   SizeOfUninitializedData;
    ULONG   AddressOfEntryPoint;
    ULONG   BaseOfCode;
    ULONG   BaseOfData;
    //
    // NT additional fields.
    //
    ULONG   ImageBase;
    ULONG   SectionAlignment;
    ULONG   FileAlignment;
    USHORT  MajorOperatingSystemVersion;
    USHORT  MinorOperatingSystemVersion;
    USHORT  MajorImageVersion;
    USHORT  MinorImageVersion;
    USHORT  MajorSubsystemVersion;
    USHORT  MinorSubsystemVersion;
    ULONG   Reserved1;
    ULONG   SizeOfImage;
    ULONG   SizeOfHeaders;
    ULONG   CheckSum;
    USHORT  Subsystem;
    USHORT  DllCharacteristics;
    ULONG   SizeOfStackReserve;
    ULONG   SizeOfStackCommit;
    ULONG   SizeOfHeapReserve;
    ULONG   SizeOfHeapCommit;
    ULONG   LoaderFlags;
    ULONG   NumberOfRvaAndSizes;
    IMAGE_DATA_DIRECTORY DataDirectory[IMAGE_NUMBEROF_DIRECTORY_ENTRIES];
    } IMAGE_OPTIONAL_HEADER, *PIMAGE_OPTIONAL_HEADER;

* Magic değeri :
 *  0x010b --> PE32
 *  0x020b --> PE32+

* SizeOfCode : Dosya içerisinde kod bulunan bölümlerin toplam size.(.text section)

* SizeOfInitializedData :İlk değeri olan verilerin bulunduğu bölümlerin toplam boyutu(.data section) 

* SizeOfUninitializedData :ilk değeri olmayan verilerin bulunduğu bölümlerin toplam boyutu(.bss section)


* AddressOfEntryPoint : Başlangıç noktasının Relative Virtual Address 'ni verir.

* BaseOfCode : Çalıştırılabilir makine kodlarını içeren .text bölümünün başlangıç Relative Virtual Address.

* BaseOfData : Statik verilerin tutulduğu .data bölümünün başlangıç Relative Virtual Address.

* ImageBase : Linker’ın PE dosyası için öncelikli tercih ettiği yüklenme adresidir.

* SectionAlignment : Dosyadaki bölümlerin hafızaya yüklenirken nasıl bir hizalamaya uyacağını belirtir. 4KB (0x00001000) değerini içerir.

* FileAlignment : Hizalama değeri bölümlerin diskte hangi değerin katına uygun olarak sıralanacağını belirtir. 512 değeri kullanılır.

* SizeOfImage : Dosyanın hafızaya yüklendiğinde ne kadar alan kaplayacağını belirtiyor.
* DllCharacteristics : DLL hakkında bazı özellikleri bitsel olarak içerir.


In [8]:
import pefile
pe = pefile.PE(pefilePath)
print("Magic : " + hex(pe.OPTIONAL_HEADER.Magic))

if hex(pe.OPTIONAL_HEADER.Magic) == '0x10b':
    print("This is a 32-bit binary")
elif hex(pe.OPTIONAL_HEADER.Magic) == '0x20b':
    print("This is a 64-bit binary")
print("SizeOfCode : " + hex(pe.OPTIONAL_HEADER.SizeOfCode))

print("SizeOfInitializedData : " + hex(pe.OPTIONAL_HEADER.SizeOfInitializedData))

print("SizeOfUninitializedData : " + hex(pe.OPTIONAL_HEADER.SizeOfUninitializedData))

print("AddressOfEntryPoint : " + hex(pe.OPTIONAL_HEADER.AddressOfEntryPoint))

print("BaseOfCode : " + hex(pe.OPTIONAL_HEADER.BaseOfCode))

print("BaseOfData : " + hex(pe.OPTIONAL_HEADER.BaseOfData))

print("ImageBase : " + hex(pe.OPTIONAL_HEADER.ImageBase))

print("SectionAlignment : " + hex(pe.OPTIONAL_HEADER.SectionAlignment))

print("FileAlignment : " + hex(pe.OPTIONAL_HEADER.FileAlignment))

print("SizeOfImage : " + hex(pe.OPTIONAL_HEADER.SizeOfImage))

print("DllCharacteristics flags : " + hex(pe.OPTIONAL_HEADER.DllCharacteristics))

print("DataDirectory: ")

print("* - " * 10)

for entry in pe.OPTIONAL_HEADER.DATA_DIRECTORY:
  print(entry.name + "\n|\n|---- Size : " + str(entry.Size) + "\n|\n|---- VirutalAddress : " + hex(entry.VirtualAddress) + '\n')    

print("* - " * 10)

Magic : 0x10b
This is a 32-bit binary
SizeOfCode : 0x1f0000
SizeOfInitializedData : 0xed000
SizeOfUninitializedData : 0x0
AddressOfEntryPoint : 0x9b37e
BaseOfCode : 0x1000
BaseOfData : 0x1f1000
ImageBase : 0x400000
SectionAlignment : 0x1000
FileAlignment : 0x1000
SizeOfImage : 0x2fc000
DllCharacteristics flags : 0x0
DataDirectory: 
* - * - * - * - * - * - * - * - * - * - 
IMAGE_DIRECTORY_ENTRY_EXPORT
|
|---- Size : 0
|
|---- VirutalAddress : 0x0

IMAGE_DIRECTORY_ENTRY_IMPORT
|
|---- Size : 340
|
|---- VirutalAddress : 0x26cdc4

IMAGE_DIRECTORY_ENTRY_RESOURCE
|
|---- Size : 400400
|
|---- VirutalAddress : 0x29a000

IMAGE_DIRECTORY_ENTRY_EXCEPTION
|
|---- Size : 0
|
|---- VirutalAddress : 0x0

IMAGE_DIRECTORY_ENTRY_SECURITY
|
|---- Size : 5496
|
|---- VirutalAddress : 0x2de000

IMAGE_DIRECTORY_ENTRY_BASERELOC
|
|---- Size : 0
|
|---- VirutalAddress : 0x0

IMAGE_DIRECTORY_ENTRY_DEBUG
|
|---- Size : 28
|
|---- VirutalAddress : 0x1f4dd0

IMAGE_DIRECTORY_ENTRY_COPYRIGHT
|
|---- Size : 0
|
|-

##### Sections Header

    typedef struct _IMAGE_SECTION_HEADER {
    UCHAR   Name[IMAGE_SIZEOF_SHORT_NAME];
    union {
            ULONG   PhysicalAddress;
            ULONG   VirtualSize;
    } Misc;
    ULONG   VirtualAddress;
    ULONG   SizeOfRawData;
    ULONG   PointerToRawData;
    ULONG   PointerToRelocations;
    ULONG   PointerToLinenumbers;
    USHORT  NumberOfRelocations;
    USHORT  NumberOfLinenumbers;
    ULONG   Characteristics;
    } IMAGE_SECTION_HEADER, *PIMAGE_SECTION_HEADER;

* .text : Çalıştırılabilir makine kodları
* .data : Global ve statik veriler, INIT.
* .rdata : Debug bilgisi 
* .bss : Global ve statik veriler, UNINIT.
* .idata : Import edilen fonksiyon bilgileri
* .edata : Export edilen fonksiyon bilgileri
* .reloc : Relocation bilgisi
* .rsrc : Resource

In [9]:
import pefile

pe = pefile.PE(pefilePath)

print("Sections Info: \n")

print("* - " * 11)

for section in pe.sections:
    print(section.Name.decode().rstrip('\x00') + "\n|\n|---- Vitual Size : " + hex(section.Misc_VirtualSize) + "\n|\n|---- VirutalAddress : " + hex(section.VirtualAddress) + "\n|\n|---- SizeOfRawData : " + hex(section.SizeOfRawData) + "\n|\n|---- PointerToRawData : " + hex(section.PointerToRawData) + "\n|\n|---- Characterisitcs : " + hex(section.Characteristics)+'\n')    

print("* - " * 11)

Sections Info: 

* - * - * - * - * - * - * - * - * - * - * - 
.text
|
|---- Vitual Size : 0x1ef2c8
|
|---- VirutalAddress : 0x1000
|
|---- SizeOfRawData : 0x1f0000
|
|---- PointerToRawData : 0x1000
|
|---- Characterisitcs : 0x60000020

.rdata
|
|---- Vitual Size : 0x7ee26
|
|---- VirutalAddress : 0x1f1000
|
|---- SizeOfRawData : 0x7f000
|
|---- PointerToRawData : 0x1f1000
|
|---- Characterisitcs : 0x40000040

.data
|
|---- Vitual Size : 0x296a8
|
|---- VirutalAddress : 0x270000
|
|---- SizeOfRawData : 0xc000
|
|---- PointerToRawData : 0x270000
|
|---- Characterisitcs : 0xc0000040

.rsrc
|
|---- Vitual Size : 0x61c10
|
|---- VirutalAddress : 0x29a000
|
|---- SizeOfRawData : 0x62000
|
|---- PointerToRawData : 0x27c000
|
|---- Characterisitcs : 0x40000040

* - * - * - * - * - * - * - * - * - * - * - 


##### Resources


* https://blog.kowalczyk.info/articles/pefileformat.html

* https://github.com/erocarrera/pefile/blob/wiki/UsageExamples.md

* https://bufferoverflows.net/exploring-pe-files-with-python/